# PaddlePaddle OCR

In [ ]:
!python3 -m pip install paddlepaddle -i https://mirror.baidu.com/pypi/simple
!pip install paddleocr

Looking in indexes: https://mirror.baidu.com/pypi/simple


In [ ]:
from paddleocr import PaddleOCR, draw_ocr

# Paddleocr目前支持的多语言语种可以通过修改lang参数进行切换
# 例如`ch`, `en`, `fr`, `german`, `korean`, `japan`
ocr = PaddleOCR(
    use_angle_cls=True, lang="en", page_num=2
)  # need to run only once to download and load model into memory


[2024/02/17 01:18:35] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=2, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='

In [ ]:
img_path = "/Users/jieyinuo/Desktop/hackathon/datasets/clear_letter.pdf"
result = ocr.ocr(img_path, cls=True)
for idx in range(len(result)):
    res = result[idx]
    for line in res:
        print(line)

[2024/02/17 01:18:45] ppocr DEBUG: dt_boxes num : 32, elapsed : 0.34914493560791016
[2024/02/17 01:18:45] ppocr DEBUG: cls num  : 32, elapsed : 0.10634398460388184
[2024/02/17 01:18:47] ppocr DEBUG: rec_res num  : 32, elapsed : 2.7287824153900146
[2024/02/17 01:18:48] ppocr DEBUG: dt_boxes num : 15, elapsed : 0.17068052291870117
[2024/02/17 01:18:48] ppocr DEBUG: cls num  : 15, elapsed : 0.05297732353210449
[2024/02/17 01:18:49] ppocr DEBUG: rec_res num  : 15, elapsed : 1.6536502838134766
[[[186.0, 253.0], [616.0, 253.0], [616.0, 275.0], [186.0, 275.0]], ('Innocence Project: Request for Assistance', 0.9859611988067627)]
[[[188.0, 305.0], [1078.0, 305.0], [1078.0, 328.0], [188.0, 328.0]], ('Directions: All requests for assistance should come directly from the applicant, when possible.', 0.9899750351905823)]
[[[188.0, 335.0], [1084.0, 335.0], [1084.0, 360.0], [188.0, 360.0]], ('Please print this form, ask the applicant to answer ALL of the following questions and send it to', 0.990713357

In [ ]:
# 显示结果
import fitz
from PIL import Image
import cv2
import numpy as np

imgs = []
with fitz.open(img_path) as pdf:
    for pg in range(0, pdf.page_count):
        page = pdf[pg]
        mat = fitz.Matrix(2, 2)
        pm = page.get_pixmap(matrix=mat, alpha=False)
        # if width or height > 2000 pixels, don't enlarge the image
        if pm.width > 2000 or pm.height > 2000:
            pm = page.get_pixmap(matrix=fitz.Matrix(1, 1), alpha=False)

        img = Image.frombytes("RGB", [pm.width, pm.height], pm.samples)
        img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        imgs.append(img)

In [ ]:
!wget -O simfang.ttf 'https://github.com/Halfish/lstm-ctc-ocr/blob/master/fonts/simfang.ttf'

--2024-02-17 01:29:15--  https://github.com/Halfish/lstm-ctc-ocr/blob/master/fonts/simfang.ttf
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4737 (4.6K) [text/plain]
Saving to: ‘simfang.ttf’

simfang.ttf         100%[===================>]   4.63K  --.-KB/s    in 0s      

2024-02-17 01:29:16 (62.3 MB/s) - ‘simfang.ttf’ saved [4737/4737]



In [ ]:
for idx in range(len(result)):
    res = result[idx]
    image = imgs[idx]
    boxes = [line[0] for line in res]
    txts = [line[1][0] for line in res]
    scores = [line[1][1] for line in res]
    # have to download simfang.ttf first, remember to change the font path
    im_show = draw_ocr(image, boxes, txts, scores, font_path="/content/simfang.ttf")
    im_show = Image.fromarray(im_show)
    im_show.save("result_page_{}.jpg".format(idx))

In [ ]:
import fitz  # PyMuPDF
from PIL import Image
import cv2
import numpy as np
import io

# Assuming 'ocr' and 'draw_ocr' functions are defined and available

def process_pdf_and_save_ocr_results(pdf_path, font_path="/content/simfang.ttf", output_pdf_path="annotated_output.pdf"):
    # Perform OCR on the PDF and annotate images
    result = ocr.ocr(pdf_path, cls=True)

    # Initialize a PDF document for the output
    output_pdf = fitz.open()

    with fitz.open(pdf_path) as pdf:
        for pg in range(pdf.page_count):
            page = pdf[pg]
            mat = fitz.Matrix(2, 2)
            pm = page.get_pixmap(matrix=mat, alpha=False)

            if pm.width > 2000 or pm.height > 2000:
                mat = fitz.Matrix(1, 1)
            pm = page.get_pixmap(matrix=mat, alpha=False)

            img = Image.frombytes("RGB", [pm.width, pm.height], pm.samples)
            image = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

            # Get OCR results for this page
            res = result[pg]
            boxes = [line[0] for line in res]
            txts = [line[1][0] for line in res]
            scores = [line[1][1] for line in res]

            im_show = draw_ocr(image, boxes, txts, scores, font_path=font_path)
            im_show_pil = Image.fromarray(cv2.cvtColor(im_show, cv2.COLOR_BGR2RGB))

            # Convert image to PDF bytes
            img_bytes = io.BytesIO()
            im_show_pil.save(img_bytes, format="PDF", resolution=100.0)
            img_bytes.seek(0)

            # Add image as a new page to the output PDF
            img_pdf = fitz.open("pdf", img_bytes.read())
            output_pdf.insert_pdf(img_pdf)

    # Save the combined PDF
    output_pdf.save(output_pdf_path)
    output_pdf.close()
    print(f"Processed PDF saved as '{output_pdf_path}'.")

In [ ]:
# Example usage
pdf_path = "/content/4.pdf"
process_pdf_and_save_ocr_results(pdf_path, output_pdf_path=pdf_path.replace(".pdf", "_annotated.pdf"))

[2024/02/17 02:21:23] ppocr DEBUG: dt_boxes num : 32, elapsed : 0.20701217651367188
[2024/02/17 02:21:23] ppocr DEBUG: cls num  : 32, elapsed : 0.07626032829284668
[2024/02/17 02:21:26] ppocr DEBUG: rec_res num  : 32, elapsed : 2.670567274093628
[2024/02/17 02:21:26] ppocr DEBUG: dt_boxes num : 10, elapsed : 0.20172667503356934
[2024/02/17 02:21:26] ppocr DEBUG: cls num  : 10, elapsed : 0.05104970932006836
[2024/02/17 02:21:27] ppocr DEBUG: rec_res num  : 10, elapsed : 0.9739923477172852
Processed PDF saved as '/content/4_annotated.pdf'.


# Visual QA

In [ ]:
!pip install uform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 69.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
from transformers import ViltProcessor, ViltForQuestionAnswering
import requests
from PIL import Image

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

In [ ]:
# import torch
# from PIL import Image

prompt = "What is the inmate number"
image = Image.open("/content/1.png").convert("RGB")
# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)
# text = "How many cats are there?"

# prepare inputs
encoding = processor(image, prompt, return_tensors="pt")

# forward pass
outputs = model(**encoding)
logits = outputs.logits
idx = logits.argmax(-1).item()
print("Predicted answer:", model.config.id2label[idx])

Predicted answer: 0
